In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'
from optimization import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


## Figure 8: Break-even times

In [ ]:
period_string = '2024-06-03_to_2025-06-01'

num_clusters = 15
cluster_labels = np.array(pd.read_csv('Data/Cluster_Results_Adaptive/cluster_labels_'+str(num_clusters)+'.csv', header=None).iloc[:,0])
vinids = np.array(pd.read_csv('Data/vinids.csv').iloc[:,1])

#make a dataframe of vinids and cluster labels
vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
label_list = np.unique(vinids_df['cluster'])

In [ ]:
def get_box_data(results_path, period_string):    
# load results into matrices
    discount_rate = 0.045
    years = 100
    low_cc = 6440 + 800 + 5505 # quasar + interconnection fee + install
    medium_cc = 6440 + 800 + 5505 + 3895# quasar + interconnection fee + install + energy management system
    high_cc = 6440 + 800 + (5505 + 3895) * 1.25 # quasar + interconnection fee + install + energy management system * 1.2 for fee in expensive area

    vinids_df = pd.DataFrame({'vinid': vinids, 'cluster': cluster_labels})
    vin_length = 749
    cost_mat_uncontrolled = []
    cost_mat_controlled = []
    cost_mat_v2g_home = []
    cost_mat_v2g_everywhere = []
    vin_not_found_idx = []

    for vin in np.arange(1,vin_length):
        try:
            cost_mat_uncontrolled.append( pd.read_csv(results_path + 'cost_uncontrolled_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_controlled.append( pd.read_csv(results_path + 'cost_no_v2g_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_home.append( pd.read_csv(results_path + 'cost_v2g_home_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
            cost_mat_v2g_everywhere.append(pd.read_csv(results_path + 'cost_v2g_everywhere_' + period_string + '_vin_' +str(vin) + '.csv').iloc[:,1])
        except:
            vin_not_found_idx.append(np.where(vinids==vin)[0])
    
    cost_mat_uncontrolled = np.array(cost_mat_uncontrolled)
    cost_mat_controlled = np.array(cost_mat_controlled)
    cost_mat_v2g_home = np.array(cost_mat_v2g_home)
    cost_mat_v2g_everywhere = np.array(cost_mat_v2g_everywhere)

    for remove in vin_not_found_idx:
        try:
            vinids_df = vinids_df.drop(vinids_df[vinids_df['vinid'] == vinids[remove[0]]].index)
        except:
            pass
    vinids_df = vinids_df.reset_index(drop=True)

    #replace nan with previous value
    for vin in np.arange(cost_mat_controlled.shape[0]):
        for week in np.arange(cost_mat_controlled.shape[1]):
            if np.isnan(cost_mat_controlled[vin, week]):
                if week == 0:
                    cost_mat_controlled[vin, week] = 0
                else:
                    cost_mat_controlled[vin, week] = cost_mat_controlled[vin, week-1]
            if np.isnan(cost_mat_uncontrolled[vin, week]):
                if week == 0:
                    cost_mat_uncontrolled[vin, week] = 0
                else:
                    cost_mat_uncontrolled[vin, week] = cost_mat_uncontrolled[vin, week-1]
            if np.isnan(cost_mat_v2g_home[vin, week]):
                if week == 0:
                    cost_mat_v2g_home[vin, week] = 0
                else:
                    cost_mat_v2g_home[vin, week] = cost_mat_v2g_home[vin, week-1]
            if np.isnan(cost_mat_v2g_everywhere[vin, week]):
                if week == 0:
                    cost_mat_v2g_everywhere[vin, week] = 0
                else:
                    cost_mat_v2g_everywhere[vin, week] = cost_mat_v2g_everywhere[vin, week-1]

    #delete rows that sum up to zero from controlled
    zero_idx = np.all(cost_mat_controlled == 0, axis=1)
    cost_mat_controlled = cost_mat_controlled[~zero_idx]
    cost_mat_uncontrolled = cost_mat_uncontrolled[~zero_idx]
    cost_mat_v2g_home = cost_mat_v2g_home[~zero_idx]
    cost_mat_v2g_everywhere = cost_mat_v2g_everywhere[~zero_idx]

    # drop the rows where zero_idx is True from vinids_df
    vinids_df = vinids_df[~zero_idx]
    vinids_df = vinids_df.reset_index(drop=True)

    num_weeks = cost_mat_uncontrolled.shape[1]
    npc_uncontrolled = np.zeros((cost_mat_uncontrolled.shape[0], num_weeks * years))
    npc_controlled = np.zeros((cost_mat_controlled.shape[0],  num_weeks * years))
    npc_v2g_low = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))
    npc_v2g_medium = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))
    npc_v2g_high = np.zeros((cost_mat_v2g_home.shape[0],  num_weeks * years))

    for year in range(years):
        for week in range(52):
            if year == 0 and week == 0:
                npc_v2g_low[:,0] = low_cc + cost_mat_v2g_home[:,0]
                npc_v2g_medium[:,0] = medium_cc + cost_mat_v2g_home[:,0]
                npc_v2g_high[:,0] = high_cc + cost_mat_v2g_home[:,0]
            else:
                npc_uncontrolled[:, year * 52 + week] = npc_uncontrolled[:, year * 52 + week-1] + cost_mat_uncontrolled[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_controlled[:, year * 52 + week] = npc_controlled[:, year * 52 + week-1] + cost_mat_controlled[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_low[:, year * 52 + week] = npc_v2g_low[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_medium[:, year * 52 + week] = npc_v2g_medium[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)
                npc_v2g_high[:, year * 52 + week] = npc_v2g_high[:, year * 52 + week-1] + cost_mat_v2g_home[:, week] / np.power(1 + discount_rate, (year * 52+week)/52)

    #find average breakeven week averaging all vehicles for low, medium and high v2g cost
    breakeven_week_low = np.where(np.nanmean(npc_uncontrolled, axis=0) > np.nanmean(npc_v2g_low, axis=0))[0]
    if breakeven_week_low.size > 0:
        breakeven_week_low = breakeven_week_low[0]
        break_low_array = np.zeros(cost_mat_uncontrolled.shape[0])
        for vin in np.arange(cost_mat_uncontrolled.shape[0]):
            try:
                break_low_array[vin] = np.where(npc_uncontrolled[vin,:] > npc_v2g_low[vin,:])[0][0]
            except:
                break_low_array[vin] = np.nan
        breakeven_week_low_std = np.nanstd(break_low_array)
    else:
        breakeven_week_low = np.nan
    breakeven_week_medium = np.where(np.mean(npc_uncontrolled, axis=0) > np.mean(npc_v2g_medium, axis=0))[0]
    if breakeven_week_medium.size > 0:
        breakeven_week_medium = breakeven_week_medium[0]
        break_medium_array = np.zeros(cost_mat_uncontrolled.shape[0])
        for vin in np.arange(cost_mat_uncontrolled.shape[0]):
            try:
                break_medium_array[vin] = np.where(npc_uncontrolled[vin,:] > npc_v2g_medium[vin,:])[0][0]
            except:
                break_medium_array[vin] = np.nan
        breakeven_week_medium_std = np.nanstd(break_medium_array)
    else:
        breakeven_week_medium = np.nan
    breakeven_week_high = np.where(np.mean(npc_uncontrolled, axis=0) > np.mean(npc_v2g_high, axis=0))[0]
    if breakeven_week_high.size > 0:    
        breakeven_week_high = breakeven_week_high[0]
        break_high_array = np.zeros(cost_mat_uncontrolled.shape[0])
        for vin in np.arange(cost_mat_uncontrolled.shape[0]):
            try:
                break_high_array[vin] = np.where(npc_uncontrolled[vin,:] > npc_v2g_high[vin,:])[0][0]
            except:
                break_high_array[vin] = np.nan
        breakeven_week_high_std = np.nanstd(break_high_array)
    else:
        breakeven_week_high = np.nan

    #find breakeven week for each vehicle for low, medium and high v2g cost
    for vin in np.arange(cost_mat_uncontrolled.shape[0]):
        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_low[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_low'] = breakeven_week

        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_medium[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_medium'] = breakeven_week    
        breakeven_idx = np.where(npc_uncontrolled[vin,:] > npc_v2g_high[vin,:])[0]
        if breakeven_idx.size > 0:
            breakeven_week = breakeven_idx[0]
        else:
            breakeven_week = np.nan
        vinids_df.loc[vin, 'breakeven_week_high'] = breakeven_week
    return vinids_df, breakeven_week_low, breakeven_week_medium, breakeven_week_high, breakeven_week_low_std, breakeven_week_medium_std, breakeven_week_high_std

In [ ]:
def plot_breakeven_times(ax, break_low, break_med, break_high, break_low_std, break_medium_std, break_high_std, break_low_aging, break_med_aging, break_high_aging, break_low_aging_std, break_med_aging_std, break_high_aging_std, circuit):
    blue_dark = "#437A98" #70A3BF'
    blue_light = "#BFE5F6"
    pink_dark = "#8D4C57" #DDCC77
    pink_light = "#FAC7C7"
    color_palette = [blue_light, pink_light,'#88CCEE',  '#CC6677',blue_dark, pink_dark]
    breakeven_dict = {
        'Low': [break_low/52, break_low_aging/52],
        'Medium': [break_low_aging/52, break_med_aging/52],
        'High': [break_high_aging/52, break_high_aging/52]
    }
    breakeven_std_dict = {
        'Low': [break_low_std/52, break_low_aging_std/52],
        'Medium': [break_low_aging_std/52, break_med_aging_std/52],
        'High': [break_high_aging_std/52, break_high_aging_std/52]
    }

    x = int(circuit[-1]) + int(circuit[-1])*0.6  # the label locations
    width = 0.2  # the width of the bars

    multiplier = 0
    for attribute, measurement in breakeven_dict.items():
        offset = width*1.2 * multiplier 
        rects0 = ax.bar(x + offset  - width/2, measurement[0], width, label=attribute, color = color_palette[multiplier])
        ax.bar_label(rects0, labels = [str(np.round(measurement[0],1))], padding=breakeven_std_dict[attribute][0]*9.5, fontsize=11)
        rects1 = ax.bar(x + offset  + width/2, measurement[1], width, label=attribute, color = color_palette[multiplier+1])
        ax.bar_label(rects1, labels = [str(np.round(measurement[1],2))], padding=breakeven_std_dict[attribute][1]*9.5, fontsize=11)

        multiplier += 2
        if attribute in breakeven_std_dict:
            ax.errorbar(x + offset -width/2, measurement[0], yerr=breakeven_std_dict[attribute][0], fmt='none', color='black', capsize=5)
            ax.errorbar(x + offset +width/2, measurement[1], yerr=breakeven_std_dict[attribute][1], fmt='none', color='black', capsize=5)

    min=-3
    max=35
    ax.set_ylim(min, max) 
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.set_ylabel('Break-even Time [years]', fontsize=15)
    if circuit == 'cir_1':
        pa1 = Patch(facecolor=blue_light, edgecolor=blue_light)
        pa2 = Patch(facecolor='#88CCEE', edgecolor='#88CCEE')
        pa3 = Patch(facecolor=blue_dark, edgecolor=blue_dark)
        #
        pb1 = Patch(facecolor=pink_light, edgecolor=pink_light)
        pb2 = Patch(facecolor='#CC6677', edgecolor='#CC6677')
        pb3 = Patch(facecolor=pink_dark, edgecolor=pink_dark)

        ax.legend(handles=[pa1, pb1, pa2, pb2, pa3, pb3],
                labels=['', '', '', '', 'Without Battery Degradation Cost', 'With Battery Degradation Cost'],
                ncol=3, handletextpad=0.5, handlelength=1.0, columnspacing=-0.5,
                loc='upper left', fontsize=16)

    coeff = 2.8
    if circuit == 'cir_1':
        x = x-0.2
        ax.set_xticks([x+width, x+coeff*width+.12, x+coeff*2*width+.03, x+coeff*3*width+.13, x+coeff*4*width+.06, x+coeff*5*width-.02, x+coeff*6*width+.05, x+coeff*7*width-.05, x+coeff*8*width-.11, x+coeff*9*width-0.03, x+coeff*10*width-.1, x+coeff*11*width-.2])
        ax.set_xticklabels(['Low', 'Medium', 'High', 'Low', 'Medium', 'High', 'Low', 'Medium', 'High', 'Low', 'Medium', 'High'])
        ax.text(0.1, -.12, 'Circuit 1', transform 
                = ax.transAxes, fontsize=16)
        ax.text(0.33, -.12, 'Circuit 2', transform
                = ax.transAxes, fontsize=16)
        ax.text(0.58, -.12, 'Circuit 3', transform
                = ax.transAxes, fontsize=16)
        ax.text(0.8, -.12, 'Circuit 4', transform
                = ax.transAxes, fontsize=16)
    return

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14, 6))
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.15)

cluster_info_df = pd.read_csv('Data/Cluster_Results_Adaptive/cluster_info_'+str(num_clusters)+'.csv')
vinids_dict = {}

for i, circuit in enumerate(['cir_1', 'cir_2', 'cir_3', 'cir_4']):
    aging = 'batt_aging_0'
    results_path = 'Results/'+ circuit + '/' + aging + '/' + 'elrp_1_'
    tmp, break_low, break_med, break_high, break_low_std, break_med_std, break_high_std = get_box_data( results_path, period_string)
    aging = 'batt_aging_99'
    results_path = 'Results/'+ circuit + '/' + aging + '/' + 'elrp_1_'
    tmp, break_low_aging, break_med_aging, break_high_aging, break_low_aging_std, break_med_aging_std, break_high_aging_std = get_box_data( results_path, period_string)
    plot_breakeven_times(ax,break_low, break_med, break_high, break_low_std, break_med_std, break_high_std, break_low_aging, break_med_aging, break_high_aging, break_low_aging_std, break_med_aging_std, break_high_aging_std, circuit)

#add line on y=0
ax.axhline(0, color='black', linewidth=0.8)
plt.savefig(save_path + '8_breakeven_times.pdf', bbox_inches='tight')
